**Table of contents**<a id='toc0_'></a>    
- [Load dữ liệu từ các thư mục train và dev](#toc1_)    
- [Pha huấn luyện](#toc2_)    
  - [Task1: Nhận diện câu so sánh](#toc2_1_)    
    - [Embbeding các câu trong dữ liệu huấn luyện](#toc2_1_1_)    
    - [Embbedding các nhãn](#toc2_1_2_)    
    - [Huấn luyện model](#toc2_1_3_)    
    - [Đánh giá model](#toc2_1_4_)    
    - [Chạy thử một vài ví dụ](#toc2_1_5_)    
  - [Task 2: Extract đầy đủ cái đặc trưng của phép so sánh.](#toc2_2_)    
    - [Tiền xử lý](#toc2_2_1_)    
    - [Huấn luyện model](#toc2_2_2_)    
- [Pha suy luận (dự đoán)](#toc3_)    
  - [Task 1: Liệu câu có so sánh hay không?](#toc3_1_)    
    - [Dự đoán nhãn](#toc3_1_1_)    
    - [Hậu xử lý](#toc3_1_2_)    
  - [Task 2: Chi tiết các vật thể được so sánh](#toc3_2_)    
    - [Dự đoán](#toc3_2_1_)    
    - [Hậu xử lý](#toc3_2_2_)    
- [Tổng hợp và lưu dữ liệu](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Load dữ liệu từ các thư mục train và dev](#toc0_)

> Dữ liệu được parse và format thành các Record (datatypes.py)

In [1]:
%load_ext autoreload
%autoreload 2

from dataloader import DataLoader

trainset = DataLoader.load_data("../data/train") + DataLoader.load_data("../data/dev")
devset = DataLoader.load_data("../data/dev")
testset = DataLoader.load_data("../data/test")
# print(f"{trainset[:3]=}")
# print(f"{devset[:3]=}")
for i in range(6):
    print(trainset[i])

Record(original='Title: iPhone 12 Pro Max được trang DisplayMate “chấm” là chiếc smartphone có màn hình đẹp nhất hiện nay', sentence='Title : iPhone 12 Pro Max được trang DisplayMate “ chấm ” là chiếc smartphone có màn hình đẹp nhất hiện nay', quintuples=[{'subject': ['3&&iPhone', '4&&12', '5&&Pro', '6&&Max'], 'object': [], 'aspect': ['17&&màn', '18&&hình'], 'predicate': ['19&&đẹp', '20&&nhất', '21&&hiện', '22&&nay'], 'label': 'SUP+'}], comparative=['SUP+'])
Record(original='alt:', sentence='alt :', quintuples=[], comparative=['No'])
Record(original='des:', sentence='des :', quintuples=[], comparative=['No'])
Record(original='Mới đây, chuyên trang đánh giá màn hình điện thoại DisplayMate đã vừa đưa ra kết luận rằng chiếc smartphone có màn hình đẹp nhất hiện nay không ai khác chính là iPhone 12 Pro Max.', sentence='Mới đây , chuyên trang đánh giá màn hình điện thoại DisplayMate đã vừa đưa ra kết luận rằng chiếc smartphone có màn hình đẹp nhất hiện nay không ai khác chính là iPhone 12 Pr

In [2]:
cnt = 0
arr_comparative = []
for record in trainset:
    if 'No' not in record.comparative :
        arr_comparative.append(cnt)
        cnt += 1

print(cnt)
print(len(trainset))

1161
5904


In [3]:
from processing import RecordEncoder
from utils import dupplicate_record_by_quintuple

cleaned_trainset = []
for record in trainset:
    if not record.quintuples:
        continue

    cleaned_trainset.extend(dupplicate_record_by_quintuple(record))
    
print(cleaned_trainset[:4])
print(len(cleaned_trainset))

[Record(original='Title: iPhone 12 Pro Max được trang DisplayMate “chấm” là chiếc smartphone có màn hình đẹp nhất hiện nay', sentence='Title : iPhone 12 Pro Max được trang DisplayMate “ chấm ” là chiếc smartphone có màn hình đẹp nhất hiện nay', quintuples=[{'subject': ['3&&iPhone', '4&&12', '5&&Pro', '6&&Max'], 'object': [], 'aspect': ['17&&màn', '18&&hình'], 'predicate': ['19&&đẹp', '20&&nhất', '21&&hiện', '22&&nay'], 'label': 'SUP+'}], comparative=['SUP+']), Record(original='Mới đây, chuyên trang đánh giá màn hình điện thoại DisplayMate đã vừa đưa ra kết luận rằng chiếc smartphone có màn hình đẹp nhất hiện nay không ai khác chính là iPhone 12 Pro Max.', sentence='Mới đây , chuyên trang đánh giá màn hình điện thoại DisplayMate đã vừa đưa ra kết luận rằng chiếc smartphone có màn hình đẹp nhất hiện nay không ai khác chính là iPhone 12 Pro Max .', quintuples=[{'subject': ['34&&iPhone', '35&&12', '36&&Pro', '37&&Max'], 'object': [], 'aspect': ['23&&màn', '24&&hình'], 'predicate': ['25&&đẹ

In [4]:
cnt = 0
for record in cleaned_trainset:
    if len(record.quintuples[0]['predicate']) == 2:
        #print(record.quintuples[0]['predicate'])
        cnt += 1
print(cnt)

509


In [5]:
cnt = 0
for record in cleaned_trainset:
    if record.quintuples[0]['subject'] == record.quintuples[0]['object']:
        cnt += 1
        print(record.quintuples[0]['subject'])
print(cnt)

['5&&các', '6&&mẫu', '7&&điện', '8&&thoại', '9&&mới', '10&&ra', '11&&mắt', '12&&của', '13&&nhà', '14&&Samsung']
[]
[]
['23&&2', '24&&chiếc', '25&&máy', '26&&này']
['23&&2', '24&&chiếc', '25&&máy', '26&&này']
['2&&hai', '3&&máy', '4&&ảnh']
[]
[]
[]
[]
[]
['10&&hai', '11&&nửa']
['14&&cả', '15&&hai']
[]
[]
[]
[]
[]
[]
[]
['1&&Cả', '2&&hai', '3&&điện', '4&&thoại']
['9&&hai', '10&&máy']
['3&&hai', '4&&chip', '5&&xử', '6&&lý']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['6&&2', '7&&mẫu', '8&&điện', '9&&thoại']
[]
['7&&hai', '8&&chiếc', '9&&máy']
[]
[]
[]
[]
[]
['1&&Cả', '2&&hai']
[]
[]
['15&&hai', '16&&model', '17&&này']
['1&&Cả', '2&&hai']
['6&&cả', '7&&hai']
[]
[]
['7&&cả', '8&&hai']
[]
[]
[]
[]
[]
['4&&cả', '5&&hai', '6&&điện', '7&&thoại']
[]
['1&&Cả', '2&&hai', '3&&điện', '4&&thoại']
['1&&Cả', '2&&hai', '3&&điện', '4&&thoại']
['1&&Cả', '2&&hai', '3&&thiết', '4&&bị']
['4&&cả', '5&&hai', '6&&điện', '7&&thoại']
['4&&cả', '5&&hai', '6&&điện', '7&&thoại']
['4&&cả', '5&&hai', '6&&điện', '7&&thoại']


# <a id='toc2_'></a>[Pha huấn luyện](#toc0_)

## <a id='toc2_1_'></a>[Task1: Nhận diện câu so sánh](#toc0_)

Phương pháp: Sử dụng model RandomForest để nhận diện đa nhãn (do mỗi câu có thể nhiều câu so sánh)
- Embbeding các câu và label
- Huấn luyện model sử dụng dữ liệu dạng vector ở bước trên

### <a id='toc2_1_1_'></a>[Embbeding các câu trong dữ liệu huấn luyện](#toc0_)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(lowercase=False, min_df=0.02, use_idf=False)

sentences = [record.sentence for record in trainset]
embeded_sentences = tfidf_vec.fit_transform(sentences)

print(embeded_sentences.shape)

(5904, 187)


In [32]:
trainset[0].comparative

['SUP+']

### <a id='toc2_1_2_'></a>[Embbedding các nhãn](#toc0_)

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

labels = [record.comparative for record in trainset]
label_classes = ["No", "DIF", "EQL", "SUP+", "SUP-", "SUP", "COM+", "COM-", "COM"]

multilabel_model = MultiLabelBinarizer(classes=label_classes)
embeded_labels = multilabel_model.fit_transform(labels)

### <a id='toc2_1_3_'></a>[Huấn luyện model](#toc0_)

In [34]:
label_classes = ["No", "DIF", "EQL", "SUP+", "SUP-", "SUP", "COM+", "COM-", "COM"]
d = {"No":0,"DIF":1,"EQL":2,"SUP+":3,"SUP-":4,"SUP":5,"COM+":6,"COM-":7,"COM":8}
zz = []
for label in labels:
    zz.append(d[label[0]])
print(zz[:5])

[3, 0, 0, 3, 3]


In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(embeded_sentences, zz)

X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.3)
#x_train, x_val, y_train, y_val = train_test_split(embeded_sentences, embeded_labels, test_size=0.2, random_state=5)


In [36]:
print(f"{x_train.shape=}")
print(f"{x_train.shape=}")
print(embeded_labels[:10])

x_train.shape=(4723, 187)
x_train.shape=(4723, 187)
[[0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]]


In [38]:
from sklearn.multioutput import MultiOutputClassifier

randomforest_model = RandomForestClassifier(n_estimators = 200, criterion = 'entropy',max_depth = 16, class_weight='balanced', random_state=0)
#multioutput_classifier = MultiOutputClassifier(randomforest_model)
#multioutput_classifier.fit(x_train, y_train)
randomforest_model.fit(X_train,y_train)
#multioutput_classifier.fit(embeded_sentences, embeded_labels)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=16, n_estimators=200, random_state=0)

In [11]:
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# from sklearn.multioutput import MultiOutputClassifier

# # Initialize individual classifiers
# catboost_classifier = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1)
# lgbm_classifier = LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100)
# rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=12)

# # Wrap individual classifiers with MultiOutputClassifier
# mo_catboost_classifier = MultiOutputClassifier(catboost_classifier)
# mo_lgbm_classifier = MultiOutputClassifier(lgbm_classifier)
# mo_rf_classifier = MultiOutputClassifier(rf_classifier)

# # Create a VotingClassifier
# voting_classifier = VotingClassifier(
#     estimators=[
#         ('catboost', mo_catboost_classifier),
#         ('lgbm', mo_lgbm_classifier),
#         ('rf', mo_rf_classifier)
#     ],
#     voting='soft'  # You can use 'hard' or 'soft' voting
# )

# # Fit the ensemble model
# voting_classifier.fit(x_train, y_train)

### <a id='toc2_1_4_'></a>[Đánh giá model](#toc0_)

In [39]:
#y_preds = multioutput_classifier.predict(x_val)
y_preds = randomforest_model.predict(X_val)

print('Accuracy = ', accuracy_score(y_val, y_preds))
report = classification_report(y_val, y_preds)
print(report)

Accuracy =  0.9761068165846802
              precision    recall  f1-score   support

           0       0.92      0.88      0.90      1432
           1       0.99      1.00      1.00      1419
           2       0.97      0.95      0.96      1451
           3       0.97      0.99      0.98      1446
           4       1.00      1.00      1.00      1424
           5       1.00      1.00      1.00      1364
           6       0.94      0.97      0.96      1472
           7       0.99      1.00      1.00      1375
           8       0.99      1.00      1.00      1424

    accuracy                           0.98     12807
   macro avg       0.98      0.98      0.98     12807
weighted avg       0.98      0.98      0.98     12807



### <a id='toc2_1_5_'></a>[Chạy thử một vài ví dụ](#toc0_)

In [13]:
sample_sentence = "des :"
embeded_sentence = tfidf_vec.transform([sample_sentence])
prediction = multioutput_classifier.predict(embeded_sentence)
print(multilabel_model.inverse_transform(prediction))
print(prediction)

[('No',)]
[[1 0 0 0 0 0 0 0 0]]


In [14]:
embeded_sentence

<1x187 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

### <a id='toc2_2_1_'></a>[Tiền xử lý](#toc0_)

In [15]:
from processing import RecordEncoder
from utils import dupplicate_record_by_quintuple

cleaned_trainset = []
for record in trainset:
    if not record.quintuples:
        continue

    cleaned_trainset.extend(dupplicate_record_by_quintuple(record))

encoder = RecordEncoder()

x = [encoder.record2features(record) for record in cleaned_trainset]
y = [encoder.record2tags(record) for record in cleaned_trainset]
# y_Sub = [encoder.record2tags_Sub(record) for record in cleaned_trainset]
# y_Obj = [encoder.record2tags_Obj(record) for record in cleaned_trainset]
# y_Asp = [encoder.record2tags_Asp(record) for record in cleaned_trainset]
# y_Pre = [encoder.record2tags_Pre(record) for record in cleaned_trainset]


In [16]:
y[0]

['5',
 '5',
 '1',
 '1',
 '1',
 '1',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '5',
 '3',
 '3',
 '4',
 '4',
 '4',
 '4']

### <a id='toc2_2_2_'></a>[Huấn luyện model](#toc0_)

In [17]:
from sklearn_crfsuite import CRF
# Error keep_tempfile but keep working
crf = CRF(
    algorithm="lbfgs",
    c1=0.05,
    c2=0.05,
    max_iterations=150,
    all_possible_transitions=False
)

try:
    crf.fit(x, y)

except: pass

# crf_Sub = CRF(
#     algorithm="lbfgs",
#     c1=0.05,
#     c2=0.05,
#     max_iterations=150,
#     all_possible_transitions=False
# )

# try:
#     crf_Sub.fit(x, y_Sub)

# except: pass

# crf_Obj = CRF(
#     algorithm="lbfgs",
#     c1=0.05,
#     c2=0.05,
#     max_iterations=150,
#     all_possible_transitions=False
# )

# try:
#     crf_Obj.fit(x, y_Obj)

# except: pass

# crf_Asp = CRF(
#     algorithm="lbfgs",
#     c1=0.05,
#     c2=0.05,
#     max_iterations=150,
#     all_possible_transitions=False
# )

# try:
#     crf_Asp.fit(x, y_Asp)

# except: pass

# crf_Pre = CRF(
#     algorithm="lbfgs",
#     c1=0.05,
#     c2=0.05,
#     max_iterations=150,
#     all_possible_transitions=False
# )

# try:
#     crf_Pre.fit(x, y_Pre)

# except: pass

# <a id='toc3_'></a>[Pha suy luận (dự đoán)](#toc0_)

## <a id='toc3_1_'></a>[Task 1: Liệu câu có so sánh hay không?](#toc0_)

### <a id='toc3_1_1_'></a>[Dự đoán nhãn](#toc0_)

In [18]:
test_embeded_sentences = tfidf_vec.transform([record.sentence for record in testset])

print(test_embeded_sentences.shape)

comparative_preditions = multioutput_classifier.predict(test_embeded_sentences)
comparative_preditions.shape


(3270, 187)


(3270, 9)

### <a id='toc3_1_2_'></a>[Hậu xử lý](#toc0_)

Decode vector về dạng string

Trong trường hợp model không đưa ra label nào, ta mặc định nó không có câu so sánh ("No")

In [19]:
readable_comparative_preditions = multilabel_model.inverse_transform(comparative_preditions)

formatted_comparative_predictions = []

for prediction in readable_comparative_preditions:
    if prediction:
        if len(prediction) == 1:
            formatted_comparative_predictions.append((prediction[0]))
        else:
            formatted_comparative_predictions.append(list(prediction)[0])
    else:
        formatted_comparative_predictions.append("No")

formatted_comparative_predictions[:30]

['No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'COM+',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'COM+',
 'No',
 'No',
 'No',
 'No',
 'No',
 'EQL',
 'No',
 'COM+',
 'No']

In [20]:
len(formatted_comparative_predictions)

3270

## <a id='toc3_2_'></a>[Task 2: Chi tiết các vật thể được so sánh](#toc0_)

### <a id='toc3_2_1_'></a>[Dự đoán](#toc0_)

In [21]:
from processing import RecordDecoder

test_featured_sentences = [encoder.record2features(record) for record in testset]
decoder = RecordDecoder()
predictions = crf.predict(test_featured_sentences)
# Sub_preditions = crf_Sub.predict(test_featured_sentences)
# Obj_preditions = crf_Obj.predict(test_featured_sentences)
# Asp_preditions = crf_Asp.predict(test_featured_sentences)
# Pre_preditions = crf_Pre.predict(test_featured_sentences)


### <a id='toc3_2_2_'></a>[Hậu xử lý](#toc0_)

In [22]:
readable_predictions = [[decoder.parse_tag(_id) for _id in postag_prediction] for postag_prediction in predictions]
# readable_Sub_predictions = [[decoder.parse_tag(_id) for _id in postag_prediction] for postag_prediction in Sub_preditions]
# readable_Obj_predictions = [[decoder.parse_tag(_id) for _id in postag_prediction] for postag_prediction in Obj_preditions]
# readable_Asp_predictions = [[decoder.parse_tag(_id) for _id in postag_prediction] for postag_prediction in Asp_preditions]
# readable_Pre_predictions = [[decoder.parse_tag(_id) for _id in postag_prediction] for postag_prediction in Pre_preditions]


In [23]:
print(readable_predictions[0:5])

[['O', 'O', 'O', 'O', 'Sub', 'Sub', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'Sub', 'Sub', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O'], ['Sub', 'Sub', 'O', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'Obj', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Sub', 'Sub', 'Sub', 'O', 'O', 'Sub', 'Sub', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


# <a id='toc4_'></a>[Tổng hợp và lưu dữ liệu](#toc0_)

In [24]:
labeled_records = [
    decoder.label_record(
        record, formatted_comparative_predictions[i], readable_predictions[i]#readable_Sub_predictions[i], readable_Obj_predictions[i],readable_Asp_predictions[i],readable_Pre_predictions[i]
    )
    for i, record in enumerate(testset)
]

for record in labeled_records[10:20]:
    print(decoder.format_record(record), end="\n\n")

Đến từ hai thương hiệu khác nhau cho nên hai chiếc điện thoại này có vẻ diện mạo hoàn toàn khác biệt nhau.	Đến từ hai thương hiệu khác nhau cho nên hai chiếc điện thoại này có vẻ diện mạo hoàn toàn khác biệt nhau .

Tuy nhiên, so sánh về thiết kế thì ưu thế sẽ nghiêng về phía iPhone 13 hơn.	Tuy nhiên , so sánh về thiết kế thì ưu thế sẽ nghiêng về phía iPhone 13 hơn .
{"subject": [], "object": [], "aspect": ["7&&thiết", "8&&kế"], "predicate": ["10&&ưu", "11&&thế"], "label": "COM+"}

Samsung Galaxy S22 sở hữu thiết kế cao cấp, mới lạ, được hoàn thiện bằng kim loại cao cấp, mặt lưng được ốp kính vô cùng tinh tế.	Samsung Galaxy S22 sở hữu thiết kế cao cấp , mới lạ , được hoàn thiện bằng kim loại cao cấp , mặt lưng được ốp kính vô cùng tinh tế .

Cụm 3 camera được xếp dọc, thẳng hàng vô cùng gọn gàng và tinh tế.	Cụm 3 camera được xếp dọc , thẳng hàng vô cùng gọn gàng và tinh tế .

Chiếc Samsung này còn được trang bị màn hình đục lỗ bắt mắt cùng viền bezel các cạnh khá mỏng tạo nên tổng thể 

In [25]:
from pathlib import Path

EXPORT_DIR = "../data/export"
DEV_DIR = "../data/dev/"
TEST_DIR = "../data/test/"
sentence2record = {
    record.original: record for record in labeled_records
}

for filepath in Path(TEST_DIR).iterdir():
    with open(filepath, "r") as fd:
        content = fd.read().strip()
        raw_records = content.split("\n\n")

        with open(Path(EXPORT_DIR, filepath.name), "w") as export_fd:
            for raw_record in raw_records:
                sentence, tokenized_sentence = raw_record.split("\t")
                record = sentence2record[sentence]
    
                print(decoder.format_record(record), end="\n\n",file=export_fd)
